# Setup & Bibliotheken

In [ ]:
'''# Schritt 1: Deinstalliere alle problematischen Pakete
!pip uninstall tensorflow tensorflow-text tf-keras tensorflow-decision-forests tensorflow-probability -y

# Schritt 2: Installiere kompatible Versionen
!pip install tensorflow==2.19.0 tensorflow-probability==0.25.0 tensorflow-text==2.19.0 tf-keras==2.19.0

!pip install -q sentence-transformers transformers accelerate torch pandas tqdm

# Schritt 3: Unterdrücke Warnungen (für parakeet und CUDA)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Unterdrückt TensorFlow-Warnungen

# Schritt 4: Teste den Import
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
print("Import erfolgreich!")

# Schritt 5: Teste tensorflow_probability
import tensorflow_probability as tfp
print(f"TensorFlow Probability Version: {tfp.__version__}")
!pip install transformers torch pandas tqdm -q'''

In [ ]:

import platform #infos aus dem System verarbeiten können (Windows)
import os #für Dateihändelung
import pandas as pd #für Datenanalyse (Bereinigung von Daten)
import random
from tqdm import tqdm #um ausgeben zu können, wie weit die Prozesse laufen (für Prozess Balken)
import tensorflow_probability as tfp # Wahrscheinlichekti von Ergebnissen analysieren & bewerten
import torch # soll dabei helfen, schnellere Berechnungen zu machen (GPU)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM #aus hugging face bibliothek, um Texte in Tokens umzuwandeln, Textsequenzen als Input und andere Textsequenzen ausgeben (Output)
#Aufgabe: Übersetzen, Zusammenfassen, Generieren
from sentence_transformers import SentenceTransformer, util #sentenceTranformer, damit werden Modelle geladen, die Texte in ganze Vektoren umwandeln
#ür Sätze/Texte erzeugen → Ähnlichkeitsvergleiche
from google.colab import files, drive #Interaktion mit lokalen Dateien in Google Colab Umgebung
from datetime import datetime# Manipulation von Zeitangaben
import shutil #Operationen mit Dateien und mit dem Sammeln von Dateien
import re #Operationen für reguläre Ausdrücke (regex)
import json
from nltk.tokenize import sent_tokenize
import nltk
# NLTK Punkt-Tokenizer
nltk.download('punkt', quiet=True)
import math
import csv
from pathlib import Path
import requests
from dotenv import load_dotenv




In [ ]:
def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

# Gewählte xlsl Datei in csv Datei umwandeln und umbenennen

In [ ]:
# title

read_file = pd.DataFrame(pd.read_excel('/content/drive/My Drive/Google_Rezensionen_qualitätsgesichert_2023_12_19.xlsx'))
read_file.to_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19.csv", index = False, header = True, encoding=smartEncoding())
df = pd.DataFrame(pd.read_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19.csv"))
#print(df.head)

In [ ]:
#wähle relevante Spalten
gewaehlte_spalten = df[["Erfahrungsbericht des Nutzers" , "Zufallszahl"]]

#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv", index = False)

# Datenbereinigung

In [ ]:
erfahrungen_gefiltert = pd.read_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_gefiltert.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_gefiltert.drop_duplicates(inplace=True)
#speichern
erfahrungen_gefiltert.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index=False)
print(f"FINAL: {len(erfahrungen_gefiltert)} Reihen")

#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)

#zufällig n = 100 Berichte für erfahrungen_final.csv auswählen
erfahrungen_clean = pd.read_csv("/content/drive/My Drive/erfahrungen_clean.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_clean.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_clean.drop_duplicates(inplace=True)

erfahrungen_clean.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)
# INDEX-Nummern zufällig wählen
zufalls_index = random.sample(range(len(erfahrungen_clean)), k=100)
zufall_100 = erfahrungen_clean.iloc[zufalls_index]
#speichern (NUR 100!)
zufall_100.to_csv("/content/drive/My Drive/zufall_100_berichte.csv", index=False)
# selben Rezensionen wurden für händische ABSA in seperaten Datei "goldstandard_clean" gespeichert


print(f"{len(zufall_100)} Zufallsberichte!")

# **Datenaugmentation**

## Datenaugmentation mit ca. 11.900 Rezensionen

## Paraphrasieren

In [ ]:

# Google Drive mounten/ Ordner in Google Colab Umbegung erstellen lassen (einmalige Ausführung nötig)
drive.mount('/content/drive')
SAVE_PATH = '/content/drive/MyDrive/Projekt_ABSA' #Unterordner "Projekt_ABSA" in Google Colab Umgebung erstellen
os.makedirs(SAVE_PATH, exist_ok=True) #Methode, wobei ein Projektverzeichnis (Pfad für Paraphrase mit inbegriffen) festgelegt wird

print("Ordner & Bibliotheken geladen!")

In [ ]:
!pip install transformers sentence-transformers nltk -q

In [ ]:
import nltk
nltk.download('punkt_tab', quiet=True)

def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

# Erstellung eines "Synonym-Dictionarys", zuerste mit Schlüsselwert wie z.B. "schmutzig" & danach Wertpaaren
SYNONYMS = {
    "Personal": ["Team", "Mitarbeiter", "Pflegekräfte", "Belegschaft", "Angestellte"],
    "Pflege": ["Betreuung", "Fürsorge", "Versorgung", "Hilfe", "Obhut"],
    "freundlich": ["nett", "höflich", "herzlich", "zuvorkommend", "liebenswürdig"],
    "unfreundlich": ["kalt", "respektlos", "rüde", "abweisend"],
    "schön": ["hübsch", "ansprechend", "gemütlich", "anheimelnd"],
    "gut": ["ordentlich", "zufriedenstellend", "positiv", "lobenswert"],
    "schlecht": ["mies", "mangelhaft", "unzureichend", "fragwürdig"],
    "Essen": ["Speisen", "Nahrung", "Mahlzeiten", "Gerichte"],
    "lecker": ["köstlich", "schmackhaft", "gut"],
    "sauber": ["gepflegt", "rein", "ordentlich", "hygienisch"],
    "hilfsbereit": ["unterstützend", "kooperativ", "engagiert", "aufmerksam"],
    "im": ["in", "bei"],
    "und": ["sowie", "–", "&"],
    "sehr": ["wirklich", "besonders", "äußerst"],
    "ist": ["bleibt", "erscheint"],
    "Haus": ["Heim", "Einrichtung"],
    "Heim": ["Haus", "Pflegeheim", "Institution", "Residenz"],
    "Mutter": ["Familienangehörige"],
    "Oma": ["Großmutter", "Omi"],
    "Opa": ["Großvater"],
    "kompetent": ["professionell", "fähig", "tüchtig"],
    "nett": ["sympathisch", "herzlich", "angenehm"],
    "Zimmer": ["Räume", "Wohnbereich", "Appartement"],
    "Bewohner": ["Senioren", "Heimbewohner", "Pflegebedürftige"],
    "Einrichtung": ["Institution", "Haus", "Pflegeheim"],
    "motiviert": ["engagiert", "eifrig", "tatkräftig"],
    "zugewandt": ["aufmerksam", "herzlich", "fürsorglich"],
    "Kommunikation": ["Austausch", "Kontakt", "Gespräche"],
    "Angehörigen": ["Familienangehörigen", "Verwandten"],
    "liebevoll": ["herzlich", "zärtlich", "fürsorglich"],
    "gepflegt": ["versorgt", "betreut", "umhegt"],
    "muss": ["soll", "hat zu sein"],
    "wenn": ["falls", "sobald"],
    "irgendetwas": ["etwas", "manchmal etwas"],
    "vermeintlich": ["angeblich", "vermeintlich", "scheinbar"],
    "nicht": ["nicht", "nicht ganz"],
    "läuft": ["funktioniert", "klappt"],
    "kann": ["darf", "ist möglich"],
    "jederzeit": ["immer", "sofort", "jederzeit"],
    "ansprechen": ["erwähnen", "anbringen", "zur Sprache bringen"],
    "gemeinsame": ["zusammen", "gemeinsam gefundene"],
    "Lösung": ["Antwort", "Ausweg"],
    "finden": ["erreichen", "entdecken"],
}

#Umdrehung des Dictionaries: Synonyme werden zu Key
CANONICAL = {}
for key, syns in SYNONYMS.items():
    key_lower = key.lower()
    CANONICAL[key_lower] = key_lower
    for s in syns:
        CANONICAL[s.lower()] = key_lower

#Synonyme ersetzen lassen
def synonym_replace(text):
    result = text
    words = set(re.findall(r'\b\w+\b', text.lower()))
    replaced = set()
    for word in words:
        if word in CANONICAL and word not in replaced:
            main = CANONICAL[word]
            candidates = [s for s in SYNONYMS.get(main.capitalize(), []) if s.lower() != word]
            if candidates:
                repl = random.choice(candidates)
                pattern = re.compile(rf'\b{re.escape(word)}\b', re.IGNORECASE)
                result = pattern.sub(
                    lambda m: repl.capitalize() if m.group(0)[0].isupper() else repl,
                    result,
                    count=1
                )
                replaced.add(word)
    return result

#Satz umstrukturieren
def restructure_sentences(text):
    if len(text) < 100:
        return text
    sents = sent_tokenize(text)
    if len(sents) <= 1:
        return text
    if random.random() < 0.75:
        if len(sents) == 2:
            return sents[1] + " " + sents[0]
        else:
            first, last = sents[0], sents[-1]
            middle = sents[1:-1]
            if middle:
                random.shuffle(middle)
            return first + " " + " ".join(middle) + " " + last
    return text

#pro Rezension 3 Paraphrasen erstellen
def generate_paraphrases(text):
    original = text.strip()
    if len(original) < 20:
        return [original] * 3

    results = set()
    attempts = 0
    max_attempts = 80

    while len(results) < 3 and attempts < max_attempts:
        step1 = synonym_replace(original)
        if len(step1) > 80 and random.random() < 0.65:
            para = restructure_sentences(step1)
        else:
            para = step1

        if para != original and len(para) > len(original) * 0.6:
            results.add(para)
        attempts += 1

    result_list = list(results)
    while len(result_list) < 3:
        result_list.append(original)
    return result_list[:3]

#Daten aus "erfahrungen_clean.csv hochladen"
input_path = "/content/drive/MyDrive/erfahrungen_clean.csv"
df = pd.read_csv(input_path)

# Sicherstellen, dass Spalte existiert
if 'Erfahrungsbericht des Nutzers' not in df.columns or 'Zufallszahl' not in df.columns:
  raise ValueError("Spalten 'Erfahrungsbericht des Nutzers' oder 'Zufallszahl' fehlen!")


df = df.dropna(subset=['Erfahrungsbericht des Nutzers']).reset_index(drop=True)
df['Zufallszahl'] = df['Zufallszahl'].astype(int)

print(f"Geladene Rezensionen: {len(df)}")
# in csv
output_path = "/content/drive/MyDrive/Projekt_ABSA/erfahrungen_paraphrased.csv"

# Liste für alle Zeilen sammeln
data_rows = []

print("Erzeuge Paraphrasen und bereite CSV vor...")

for _, row in tqdm(df.iterrows(), total=len(df), desc="Paraphrasen erstellen"):
    original_id = int(row['Zufallszahl'])
    original_text = str(row['Erfahrungsbericht des Nutzers']).strip()

    # 1. Original eintragen
    data_rows.append({
        'id': original_id,
        'text': original_text,
        'typ': 'original'
    })

    # 2. Drei Paraphrasen erzeugen und eintragen
    paraphrases = generate_paraphrases(original_text)
    for i, para_text in enumerate(paraphrases, 1):
        data_rows.append({
            'id': original_id,
            'text': para_text.strip(),
            'typ': f'paraphrase_{i}'
        })

# In DataFrame umwandeln und als CSV speichern
result_df = pd.DataFrame(data_rows)

# Reihenfolge der Spalten festlegen
result_df = result_df[['id', 'text', 'typ']]

# Speichern mit smartEncoding()
result_df.to_csv(output_path, index=False, encoding=smartEncoding())

print(f"\nFERTIG! CSV erfolgreich gespeichert:")
print(f"   → {output_path}")
print(f"   → {len(df)} Originale + {len(df)*3} Paraphrasen = {len(result_df)} Zeilen insgesamt")

# Optional: direkter Download im Colab
from google.colab import files
print("\nStarte Download der Datei...")
files.download(output_path)
